In [1]:
import re
import json
import requests
from datetime import datetime
from typing import Dict, TypedDict, Any, Tuple
import numpy as np
import boto3
import rasterio
from rasterio.io import MemoryFile
from rasterio.mask import mask
from rasterio.merge import merge
from rasterio.session import AWSSession
from pystac_client import Client
import planetary_computer
from planetary_computer import sign
from shapely.geometry import shape
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda
from langchain.tools import tool


def parse_user_message(state: dict) -> dict:
    """
    Parse user message from `state["message"]`, returning aoi, date_range, and thresholds.
    """
    message = state["message"]
    geojson_match = re.search(r'\{.*\}', message, re.DOTALL)
    geojson_str = geojson_match.group(0) if geojson_match else None
    geojson = json.loads(geojson_str)

    date_match = re.search(r'date range:\s*(\d{4}-\d{2}-\d{2})\s*to\s*(\d{4}-\d{2}-\d{2})', message)
    start_date, end_date = date_match.groups()

    conf_match = re.search(r'confidence:\s*([0-9.]+)', message)
    intensity_match = re.search(r'intensity:\s*([0-9.]+)', message)

    return {
        "aoi": geojson,
        "date_range": (start_date, end_date),
        "thresholds": {
            "confidence": float(conf_match.group(1)),
            "intensity": int(float(intensity_match.group(1)))
        }
    }


In [56]:
ESA_CLASS_NAMES = {
    0: "No Data",
    10: "Tree cover",
    20: "Shrubland",
    30: "Grassland",
    40: "Cropland",
    50: "Built-up",
    60: "Bare / sparse vegetation",
    70: "Snow and ice",
    80: "Permanent water bodies",
    90: "Herbaceous wetland",
    95: "Mangroves",
    100: "Moss and lichen"
}

def filter_and_count_pixels(inputs: Dict) -> Dict:
    """Filter COG pixels and count results per ESA WorldCover land cover class."""
    aoi = inputs["aoi"]
    date_range = inputs["date_range"]
    thresholds = inputs["thresholds"]

    start_date = datetime.strptime(date_range[0], "%Y-%m-%d")
    end_date = datetime.strptime(date_range[1], "%Y-%m-%d")
    start_days = (start_date - datetime(2015, 1, 1)).days
    end_days = (end_date - datetime(2015, 1, 1)).days

    default_path = "s3://gfw-data-lake/umd_glad_dist_alerts/v20250329/raster/epsg-4326/cog/default.tif"
    intensity_path = "s3://gfw-data-lake/umd_glad_dist_alerts/v20250329/raster/epsg-4326/cog/intensity.tif"

    session = boto3.Session(profile_name="zeno_internal_sso")
    geometry = [shape(aoi["geometry"]).__geo_interface__]

    with rasterio.Env(AWSSession(session), AWS_REQUEST_PAYER="requester"):
        with rasterio.open(default_path) as src1:
            default_data, _ = mask(src1, geometry, crop=True)
        with rasterio.open(intensity_path) as src2:
            intensity_data, _ = mask(src2, geometry, crop=True)

    encoded = default_data[0]
    confidence = encoded // 10000
    days_since_2015 = encoded % 10000
    intensities = intensity_data[0]

    print("requested start and end days since 2015: ", start_days, end_days)

    # Search ESA WorldCover STAC API for all intersecting items
    stac = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    aoi_geom = shape(aoi["geometry"])

    start_search_date = start_date.date().isoformat()
    end_search_date = end_date.date().isoformat()

    search = stac.search(
        collections=["esa-worldcover"],
        intersects=aoi_geom,
        datetime=f"{start_search_date}/{end_search_date}",
        max_items=10
    )
    items = list(search.get_items())
    

    if not items:
        raise ValueError("No ESA WorldCover items found for AOI.")

    # Sign and open all map assets, collect datasets
    asset_urls = []
    for item in items:
        if "map" in item.assets:
            asset_urls.append(sign(item.assets["map"].href))

    with rasterio.Env(AWSSession(session), AWS_REQUEST_PAYER="requester"):
        datasets = [rasterio.open(url) for url in asset_urls]
        # Mosaic all ESA WorldCover tiles covering the AOI
        mosaic, out_transform = merge(datasets)
        # Create an in-memory rasterio dataset for the mosaic for masking
        profile = datasets[0].profile.copy()
        profile.update({
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_transform
        })
        with rasterio.io.MemoryFile() as memfile:
            with memfile.open(**profile) as dataset:
                dataset.write(mosaic)
                # Mask mosaic by AOI geometry
                land_cover_data, _ = mask(dataset, geometry, crop=True)
        for ds in datasets:
            ds.close()

    land_cover = land_cover_data[0]

    # Align shapes to smallest common shape
    min_rows = min(default_data.squeeze().shape[0], land_cover.shape[0])
    min_cols = min(default_data.squeeze().shape[1], land_cover.shape[1])

    default_data = default_data.squeeze()[:min_rows, :min_cols]
    land_cover = land_cover[:min_rows, :min_cols]
    confidence = confidence[:min_rows, :min_cols]
    days_since_2015 = days_since_2015[:min_rows, :min_cols]
    intensities = intensities[:min_rows, :min_cols]

    if land_cover.shape != confidence.shape:
        raise ValueError(f"Shape mismatch: land cover {land_cover.shape} vs alerts {confidence.shape}")

    reference_date = datetime(2015, 1, 1)
    today = datetime.today()
    max_days = (today - reference_date).days

    valid_mask = (
        (confidence >= thresholds["confidence"]) &
        (days_since_2015 >= start_days) &
        (days_since_2015 <= end_days) &
        (days_since_2015 < max_days) &
        (intensities >= thresholds["intensity"])
    )

    # Count pixels per ESA class
    results = {}
    valid_lc = land_cover[valid_mask]
    unique_classes, counts = np.unique(valid_lc, return_counts=True)

    for cls, count in zip(unique_classes, counts):
        results[int(cls)] = {
            "class": ESA_CLASS_NAMES.get(int(cls), "Unknown"),
            "count": int(count)
        }

    return {"result": results}



In [57]:
class GraphState(TypedDict):
    message: str
    aoi: dict
    date_range: Tuple[str, str]
    thresholds: Dict[str, float]
    result: Dict[str, Any]

builder = StateGraph(GraphState)

builder.add_node("parse_message", RunnableLambda(parse_user_message))
builder.add_node("filter_and_count", RunnableLambda(filter_and_count_pixels))

builder.set_entry_point("parse_message")
builder.add_edge("parse_message", "filter_and_count")
builder.add_edge("filter_and_count", END)

graph = builder.compile()


In [58]:
#2 is low confidence, 3 is high confidence (1 means nothing and the values should all be >=20,000 and <40,000)

In [59]:
# Test AOI over the Peruvian Amazon (Loreto and Ucayali regions)

message = """
{
  "type": "Feature",
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [-75.0, -3.0],
        [-70.0, -3.0],
        [-70.0, -6.0],
        [-75.0, -6.0],
        [-75.0, -3.0]
      ]
    ]
  }
}
date range: 2016-01-01 to 2020-12-31
confidence: 3
intensity: 50
"""

result = graph.invoke({"message": message})
print(result)



requested start and end days since 2015:  365 2191


/Users/lilliannethomas/Desktop/work/wri/project-zeno/.venv/lib/python3.11/site-packages/pystac_client/item_search.py:888: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


{'message': '\n{\n  "type": "Feature",\n  "geometry": {\n    "type": "Polygon",\n    "coordinates": [\n      [\n        [-75.0, -3.0],\n        [-70.0, -3.0],\n        [-70.0, -6.0],\n        [-75.0, -6.0],\n        [-75.0, -3.0]\n      ]\n    ]\n  }\n}\ndate range: 2016-01-01 to 2020-12-31\nconfidence: 3\nintensity: 50\n', 'aoi': {'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-75.0, -3.0], [-70.0, -3.0], [-70.0, -6.0], [-75.0, -6.0], [-75.0, -3.0]]]}}, 'date_range': ('2016-01-01', '2020-12-31'), 'thresholds': {'confidence': 3.0, 'intensity': 50}, 'result': {10: {'class': 'Tree cover', 'count': 492101}, 20: {'class': 'Shrubland', 'count': 73}, 30: {'class': 'Grassland', 'count': 478}, 60: {'class': 'Bare / sparse vegetation', 'count': 52}, 80: {'class': 'Permanent water bodies', 'count': 2863}}}
